In [1]:
import cv2
import mediapipe as mp
import pyttsx3
import time
import math
import threading

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

counter = 0
stage = "center"
hold_start_time = None
hold_duration = 5
position_confirm_time = 1
position_time = None
warned = False
last_spoken_second = None

target_reps = int(input("Enter number of reps: "))

engine = pyttsx3.init()

def speak(text):
    threading.Thread(target=lambda: (engine.say(text), engine.runAndWait())).start()

def calculate_angle(a, b, c):
    ba = [a[0] - b[0], a[1] - b[1]]
    bc = [c[0] - b[0], c[1] - b[1]]
    dot_product = ba[0]*bc[0] + ba[1]*bc[1]
    magnitude_ba = math.hypot(ba[0], ba[1])
    magnitude_bc = math.hypot(bc[0], bc[1])
    angle = math.acos(dot_product / (magnitude_ba * magnitude_bc + 1e-8))
    return math.degrees(angle)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                    landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            left_ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
            right_ear = [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y]

            angle = calculate_angle(right_ear, nose, left_ear)

            current_time = time.time()

            if angle < 30:
                if nose[0] > left_ear[0] and stage != "right":
                    stage = "right"
                    position_time = current_time
                    speak("Hold for five seconds")
                    warned = False
                    last_spoken_second = None

                elif nose[0] < right_ear[0] and stage != "left":
                    stage = "left"
                    position_time = current_time
                    speak("Hold for five seconds")
                    warned = False
                    last_spoken_second = None

            if position_time:
                if current_time - position_time >= position_confirm_time:
                    if hold_start_time is None:
                        hold_start_time = current_time

                    # تأكد أن المستخدم لا يزال في نفس الوضعية بناءً على الزاوية
                    still_holding = angle < 30 and (
                        (stage == "right" and nose[0] > left_ear[0]) or
                        (stage == "left" and nose[0] < right_ear[0])
                    )

                    if still_holding:
                        elapsed = current_time - hold_start_time
                        remaining = math.ceil(hold_duration - elapsed)

                        if remaining > 0 and remaining != last_spoken_second:
                            speak(str(remaining))
                            last_spoken_second = remaining

                        if elapsed >= hold_duration:
                            counter += 1
                            speak(str(counter))
                            hold_start_time = None
                            stage = "center"
                            position_time = None
                            last_spoken_second = None

                            if counter >= target_reps:
                                message = "Great job! You smashed your goal"
                                cv2.putText(image, message, (50, 200),
                                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3, cv2.LINE_AA)
                                speak(message)
                                time.sleep(5)
                                break
                        else:
                            cv2.putText(image, f"Hold: {remaining}",
                                        (400, 100), cv2.FONT_HERSHEY_SIMPLEX,
                                        2, (0, 0, 255), 4, cv2.LINE_AA)
                    else:
                        if not warned:
                            speak("You moved, hold again")
                            warned = True
                        position_time = None
                        hold_start_time = None
                        last_spoken_second = None

        except:
            pass

        cv2.rectangle(image, (0, 0), (250, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Neck Stretch Tracker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Exception in thread Thread-4 (<lambda>):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_29832\2932695642.py", line 27, in <lambda>
  File "C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')